In [17]:
import os
import torch 
import pytorch_lightning as pl #pip install pytorch-lightning
from monai.networks.nets import UNet # pip install monai
from monai.losses import DiceLoss
from monai.data import CacheDataset, DataLoader
from monai.transforms import Compose, RandSpatialCrop, RandRotate90, ToTensor
import numpy as np
import nibabel as nib # pip install nibabel
import nrrd

In [19]:
# Version-1 

class AutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.unet = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2, #########################################
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
        )
        self.loss_function = DiceLoss(sigmoid=True, to_onehot_y=True)

    def forward(self, x):
        return self.unet(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self.unet(x)
        loss = self.loss_function(x_hat, x)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Define dataset and data loader
data_dir = "./images"
transform = Compose([
    RandSpatialCrop((256, 256, 256), random_size=False),  # Ensure the crop size is exactly (256, 256, 256)
    RandRotate90(prob=0.5),  # Rotate randomly by 90 degrees with a probability of 0.5
    ToTensor()
])

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.nrrd')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        
        # Use pynrrd to load NRRD file
        data, header = nrrd.read(file_path)
        
        # Assuming data is already in the desired format (e.g., numpy array)
        if self.transform:
            data = self.transform(data)
        
        # Assuming target is the same as input for autoencoder
        return data, data


dataset = MyDataset(data_dir, transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0) #ADJUSTED NUM_WORKERS TO 0 and solved runtime error

# Define Lightning trainer
trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor='train_loss')], ##################################
    profiler='simple'
)


# Initialize model and start training
model = AutoEncoder()
trainer.fit(model, dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type     | Params
-------------------------------------------
0 | unet          | UNet     | 4.8 M 
1 | loss_function | DiceLoss | 0     
-------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.226    Total estimated model params size (MB)
c:\Users\tgafrick\Downloads\MESAL-Image\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Users\tgafrick\Downloads\MESAL-Image\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 

RuntimeError: applying transform <monai.transforms.croppad.array.RandSpatialCrop object at 0x0000026BA0C13C90>